# Eliminación de preguntas que no cumplen los criterios
Con el dataset limpio, podemos comenzar a realizar un análisis utilizando algunas técnicas de NLP para eliminar aquellas preguntas que no cumplan con los criterios del INE establecidos en el punto

El resultado de este paso serán:
- Un archivo que contenga el texto de las preguntas que no cumplen los criterios y una breve explicación de porqué no está cumpliéndolos (discurso de odio, violencia, malas palabras, etc.)

- Un archivo con las preguntas restantes (los campos serán los mismos que en el punto 3.1)

- ID del registro

- Entidad de Origen

- Edad

- Género

- Identificación con algún grupo en situación de discriminación

- Tema de la pregunta

- Texto de la pregunta

# Proceso

1. Importamos los módulos necesarios y leemos el nuevo DF

In [250]:
from pysentimiento import create_analyzer
from math import nan
from utils import save_df
import pandas as pd
import nltk

limpio_df = pd.read_csv('./out_datasets/1_base_limpia.csv')
limpio_df.isna().sum()

entidad                 0
edad                    0
genero                  0
grupo_discriminacion    0
tema                    0
pregunta                0
fecha                   0
dtype: int64

## Remover preguntas con palabras invalidas
Debido a que el análisis de odio tarda mucho (~4min) primero vamos hacer el dataset un poco mas chico

In [268]:
palabras_invalidas = pd.read_csv('./in_datasets/palabras_baneadas.csv')

# Pasamos todo el diccionario a minúsculas
palabras_invalidas['palabra'] = palabras_invalidas['palabra'].apply(lambda x: x.lower())

Hacemos una función que cheque si las preguntas contienen palabras baneadas, si es asi se agrega un diccionario con la palabra y el motivo de su baneo.

In [252]:
def has_invalid_words(pregunta: str, *, palabras_invalidas: pd.DataFrame) -> bool:
    # Solo se normaliza la pregunta (pasamos a lower case) ya que se asume que el diccionario de palabras baneadas cubre todos los edge-cases
    pregunta = pregunta.lower()
    pregunta = nltk.word_tokenize(pregunta)
    palabras_ban = list(palabras_invalidas['palabra'])
    for palabra in pregunta:
        if palabra in palabras_ban:
            index = palabras_ban.index(palabra)
            return { 'palabra': palabra, 'motivo': palabras_invalidas.loc[index, 'categoría'] }
    return nan

In [253]:
limpio_df['ban'] = limpio_df['pregunta'].apply(has_invalid_words, palabras_invalidas=palabras_invalidas)

Generamos un DF con las preguntas baneadas y procedemos a guardarlo con solo la información relevante

In [254]:
preguntas_ban = limpio_df.dropna()
preguntas_ban = preguntas_ban[['pregunta', 'ban']]
preguntas_ban['motivo'] = preguntas_ban['ban'].apply(lambda dict: dict['motivo'])
preguntas_ban['palabra'] = preguntas_ban['ban'].apply(lambda dict: dict['palabra'])
preguntas_ban = preguntas_ban.drop(['ban'], axis=1)
save_df(preguntas_ban, "2-0_preguntas_baneadas.csv", index=False)
print(f"Se banearon {preguntas_ban.shape[0]} preguntas.")

Se banearon 991 preguntas.


## Remover preguntas con discurso de odio

In [255]:
limpio_df = limpio_df[limpio_df['ban'].isna()]
limpio_df = limpio_df.drop('ban', axis=1)
limpio_df

,entidad,edad,genero,grupo_discriminacion,tema,pregunta,fecha
0,Ciudad de México,43,Femenino,No Aplica,No discriminación y grupos vulnerables,"Candidat@, cómo apoyaría a los productores de ...",2024-03-22 00:04:00
1,Aguascalientes,14,Masculino,No Aplica,Salud,En México muchas personas se quejan del sistem...,2024-03-22 00:04:00
2,Nuevo León,17,Masculino,Personas de la diversidad sexual,Educación,¿Qué estrategias y planes concretos se propone...,2024-03-22 00:02:00
3,México,34,Femenino,No Aplica,Salud,Dentro de su plan de gobierno ¿qué acciones es...,2024-03-22 00:01:00
4,México,58,Masculino,Personas con discapacidad,Combate a la corrupción,"Por qué ,sigue la corrupción a pesar se, saber...",2024-03-21 23:59:00
...,...,...,...,...,...,...,...
22885,Morelos,69,Masculino,Personas adultas mayores,Transparencia,"La administracion y control, de puertos, aerop...",2024-02-20 19:47:00
22886,Sonora,44,Femenino,Personas afromexicanas,No discriminación y grupos vulnerables,Que hará para poder favorecer a los mexicanos ...,2024-02-20 12:27:00
22887,Guanajuato,25,Masculino,No Aplica,Violencia en contra de las mujeres,Se le cerraran las puertas a las mujeres en pa...,2024-02-20 11:59:00
22888,Puebla,24,Masculino,No Aplica,No discriminación y grupos vulnerables,Qué apoyo se les brindará a los grupos más mar...,2024-02-20 11:41:00


Creamos el analizador de odio

In [256]:
hate_speech_analyzer = create_analyzer(task="hate_speech", lang="es")

/Users/yo-lo-pregunto/miniconda3/envs/NLP/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


Definimos una funcion para determinar si la pregunta tiene discurso de odio o no en base a un umbral

In [257]:
UMBRAL = 0.25
def has_hate_speech(pregunta: str) -> bool:
   analysis = hate_speech_analyzer.predict(pregunta)
   rate = sum(analysis.probas.values()) / len(analysis.probas)
   return analysis if rate >= UMBRAL else nan
    

In [258]:
limpio_df['hate_speech'] = limpio_df['pregunta'].apply(has_hate_speech)
limpio_df

,entidad,edad,genero,grupo_discriminacion,tema,pregunta,fecha,hate_speech
0,Ciudad de México,43,Femenino,No Aplica,No discriminación y grupos vulnerables,"Candidat@, cómo apoyaría a los productores de ...",2024-03-22 00:04:00,NaN
1,Aguascalientes,14,Masculino,No Aplica,Salud,En México muchas personas se quejan del sistem...,2024-03-22 00:04:00,NaN
2,Nuevo León,17,Masculino,Personas de la diversidad sexual,Educación,¿Qué estrategias y planes concretos se propone...,2024-03-22 00:02:00,NaN
3,México,34,Femenino,No Aplica,Salud,Dentro de su plan de gobierno ¿qué acciones es...,2024-03-22 00:01:00,NaN
4,México,58,Masculino,Personas con discapacidad,Combate a la corrupción,"Por qué ,sigue la corrupción a pesar se, saber...",2024-03-21 23:59:00,NaN
...,...,...,...,...,...,...,...,...
22885,Morelos,69,Masculino,Personas adultas mayores,Transparencia,"La administracion y control, de puertos, aerop...",2024-02-20 19:47:00,NaN
22886,Sonora,44,Femenino,Personas afromexicanas,No discriminación y grupos vulnerables,Que hará para poder favorecer a los mexicanos ...,2024-02-20 12:27:00,NaN
22887,Guanajuato,25,Masculino,No Aplica,Violencia en contra de las mujeres,Se le cerraran las puertas a las mujeres en pa...,2024-02-20 11:59:00,NaN
22888,Puebla,24,Masculino,No Aplica,No discriminación y grupos vulnerables,Qué apoyo se les brindará a los grupos más mar...,2024-02-20 11:41:00,NaN


Procesamiento las preguntas con discurso de odio y las guardamos junto con el motivo de su eliminación

In [259]:
hate_df = limpio_df.dropna()

Escribimos los motivos de la eliminación de la pregunta

In [260]:
hate_df = hate_df[["pregunta", "hate_speech"]]
hate_df['motivo'] = hate_df['hate_speech'].apply(lambda analysis: ", ".join(hate for hate in analysis.output))
hate_df['ponderacion'] = hate_df['hate_speech'].apply(lambda analysis: round(sum(analysis.probas.values())/len(analysis.probas), 2))
hate_df = hate_df.drop('hate_speech', axis=1)

Se guardan las preguntas eliminadas

In [262]:
save_df(hate_df, "2-1_hate_speech.csv", index=False)
print(f"Se eliminaron {hate_df.shape[0]} preguntas relacionadas al odio")

Se eliminaron 336 preguntas relacionadas al odio


Finalmente se guardan las preguntas que no tienen ningún discurso de odio ni palabras baneadas.

In [266]:
limpio_df = limpio_df[limpio_df['hate_speech'].isna()]
print(f"Escribiendo {limpio_df.shape[0]} preguntas restantes")
save_df(limpio_df, "2-2_base_limpia.csv")

Escribiendo 21563 preguntas restantes
